# Klasifikácia pomocou zložených modelov - Random Forests

Random Forests predstavujú v súčasnosti jeden z najpoužívanejších klasifikačných modelov. Sciki-learn obsahuje implementáciu tohoto algoritmu v triede `RandomForestClassifier`. Tento klasifikátor sa používa rovnakým spôsobom, ako ostatné klasifikátory.

Opäť načítame dáta ako v predošlých príkladoch. Tentoraz ale pre algoritmus Random Forests neodstránime redundantné atribúty a ponecháme aj nominálny atribút `Deck` (aj s chýbajúcimi hodnotami) a transformujeme ho pomocou One Hot Encoderu. 

In [8]:
# Titanic import a preprocessing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

titanic = pd.read_csv("../data/titanic-processed.csv")

titanic = titanic.drop(columns=['ticket', 'cabin'])

titanic['sex'] = titanic['sex'].map({"male": 0, "female": 1})
titanic['has_family'] = titanic['has_family'].map({False: 0, True: 1})

titanic['fare_ordinal'] = titanic['fare_ordinal'].map({"normal": 0, "more expensive": 1, "most expensive": 2})
titanic['age_ordinal'] = titanic['age_ordinal'].map({"child": 0, "young": 1, "adult": 2, "old": 3}) 

titanic = pd.get_dummies(titanic, columns=['embarked', 'title_short', 'deck', 'title'])

titanic.head()

,pclass,survived,sex,age,sibsp,parch,fare,family,has_family,fare_ordinal,...,title_Master,title_Miss,title_Mlle,title_Mme,title_Mr,title_Mrs,title_Ms,title_Rev,title_Sir,title_the Countess
0,1,1,1,29.00,0,0,211.3375,0,0,2,...,0,1,0,0,0,0,0,0,0,0
1,1,1,0,0.92,1,2,151.5500,3,1,2,...,1,0,0,0,0,0,0,0,0,0
2,1,0,1,2.00,1,2,151.5500,3,1,2,...,0,1,0,0,0,0,0,0,0,0
3,1,0,0,30.00,1,2,151.5500,3,1,2,...,0,0,0,0,1,0,0,0,0,0
4,1,0,1,25.00,1,2,151.5500,3,1,2,...,0,0,0,0,0,1,0,0,0,0


Model môžeme upravovať viacerými parametrami. Keďže sa jedná o model, ktorý tvorí množstvo rôznych stromových klasifikátorov na rôznych podmnožinách vstupných dát, väčšina parametrov bude identická s rozhodovacími stromami: 
* `n_estimators` - počet stromov v "lese"
* `bootstrap` - 
* `oob_score` - True/False - či používať alebo nie out-of-bag príklady na odhad presnosti
* `criterion` - kritérium pre voľbu atribúrov - "gini", "entropy"
* `max_depth` - maximálna hĺbka stromu (ak je nastavená na None, expanduje sa plný strom)
* `min_samples_split` - najmenší počet príkladov potrebných pre vetvenie uzlu
* `min_samples_leaf` - najmenší možný počet príkladov v listovom uzle

In [9]:
X_titanic = titanic.drop('survived', axis=1) # vytvoríme maticu príznakov - použijeme všetky stĺpce okrem cieľového atribútu a uložíme do X_titanic
y_titanic = titanic['survived'] # vytvoríme vektor hodnôt cieľového atribútu ako stĺpec 'survived'

print(X_titanic.shape) # pre kontrolu môžeme vypísať rozmery matice hodnôt a vektora cieľového atribútu
print(y_titanic.shape)

from sklearn.model_selection import train_test_split # importujeme funkciu train_test_split()
X_train, X_test, y_train, y_test = train_test_split(X_titanic, y_titanic, test_size=0.3, random_state=1) # rozdelíme dataset do trénovacej a testovacej časti, tak že testovacia bude 30% z celkového datasetu

(1309, 43)
(1309,)


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

rf = RandomForestClassifier(n_estimators=500, max_depth=3)
rf.fit(X_train, y_train)

y_rf = rf.predict(X_test)

print(f"Presnosť (accuracy) modelu: {accuracy_score(y_test, y_rf)}")
cm = confusion_matrix(y_test, y_rf)
print(cm)

Presnosť (accuracy) modelu: 0.8091603053435115
[[203  33]
 [ 42 115]]


Dôležitosť atribútov je podstatný výstup z modelu Random Forest. K dôležitosti atribútov sa dostaneme pomocou `feature_importances_`, ktoré sú súčasťou modelu Random Forest. Môžeme ich usporiadať a vypísať aj s jednotlivými zodpovedajúcimi názvami atribútov.

In [11]:
sorted(zip(rf.feature_importances_, X_train.columns), reverse=True)

[(0.18135596991559422, 'sex'),
 (0.1689970781236542, 'title_short_Mr'),
 (0.14611589399782904, 'title_Mr'),
 (0.07301634728407837, 'fare'),
 (0.0728181218658206, 'pclass'),
 (0.05819709858853442, 'title_Mrs'),
 (0.05245514305825605, 'title_short_Mrs'),
 (0.04318189144226366, 'title_short_Miss'),
 (0.03729185149339787, 'title_Miss'),
 (0.031087641642430212, 'fare_ordinal'),
 (0.02713352586661763, 'family'),
 (0.018831662695872744, 'age'),
 (0.015087889758640413, 'sibsp'),
 (0.009545188080110458, 'embarked_Cherbourg'),
 (0.007863886357412683, 'parch'),
 (0.007536877730958542, 'age_ordinal'),
 (0.007198688526580262, 'deck_B'),
 (0.006754551553698402, 'has_family'),
 (0.00670158920148308, 'deck_E'),
 (0.0064818320026560385, 'embarked_Southampton'),
 (0.004450723966340371, 'title_Master'),
 (0.0036878701213947716, 'title_short_rare title'),
 (0.003334692610729598, 'deck_C'),
 (0.002615011677371509, 'deck_D'),
 (0.00190054468684585, 'embarked_Queenstown'),
 (0.0018430540432253627, 'deck_A'),

#### Úloha 12.5.

Vyskúšajte natrénovať model Random Forests na dátach Titanic. 
Vyskúšajte rôzne hodnoty parametrov (najmä `n_estimators` nastaviť na rádovo rozdielne hodnoty, napr. 10, 100, 1000). 
Použite ale rôzne nastavenia parametrov stromov - porovnajte Random Forest s nastaveniami stromov, ktoré vyplynuli ako optimálne z predchádzajúcej úlohy. Porovnajte výsledky s Random Forest s takými stromami, ktoré obsahuju veľa plytkých stromov. Líšia sa nejako? 

In [32]:
# YOUR CODE HERE


from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

# vytvorime Random Forests s 1000 stromami, ktore budu mat maximalnu hlbku 4, ktoru sme identifikovali pri skumani stromovych klasifikatorov
rf = RandomForestClassifier(n_estimators=1000, max_depth=4)
rf.fit(X_train, y_train) # natrenujeme model na trenovacej mnozine

y_rf = rf.predict(X_test) # vypocitame predikcie testovacich dat

# vypocitame a vypiseme metriky accuracy, precision, recall a F1, na zaver aj confusion matrix
print("Accuracy: %0.3f" % accuracy_score(y_test, y_rf))
print("Precision: %0.3f" % precision_score(y_test, y_rf))
print("Recall: %0.3f" % recall_score(y_test, y_rf))
print("F1: %0.3f" % f1_score(y_test, y_rf))
print()
cm = confusion_matrix(y_test, y_rf)
print(cm)

# po preskumani vysledkov mozme zhodnotit, ze Random Forests - vytvorili robustnejsi model ako samotne stromy
# aj ked presnost je zhruba rovnaka, oproti stromovym modelom sa vyrazne zlepsila navratnost

Accuracy: 0.814
Precision: 0.784
Recall: 0.739
F1: 0.761

[[204  32]
 [ 41 116]]
